In [ ]:
## Setup

import boto3

source_region = 'us-west-2'
dest_region = 'us-east-1'
db_name = 'dsdb'

source_client = boto3.client('rds',region_name=source_region)
dest_client = boto3.client('rds',region_name=dest_region)

In [ ]:
## Can we find the 'droids we're looking for?
id = 'not-found'
response = source_client.describe_db_instances()
instances = response['DBInstances']
for i in instances:
    if i['DBName'] == db_name.upper():
        id = i['DBInstanceIdentifier']
        break

if id == 'not-found':
    raise ValueError('Database ' + db_name + ' not found in region ' + source_region)
else:
    print 'db id is ' + id

In [ ]:
response = source_client.describe_db_snapshots(
     DBInstanceIdentifier=id
)

source_snapshots = []
snapshots = response['DBSnapshots']
for s in snapshots:
    if s['Status'] != 'available':
        continue
    source = {}
    source['id'] = s['DBSnapshotIdentifier']
    source['arn'] = s['DBSnapshotArn']
    source['state'] = s['Status']
    source_snapshots.append(source)
    
print source_snapshots


In [ ]:
response = dest_client.describe_db_snapshots(
     DBInstanceIdentifier=id
)

dest_snapshots = []
snapshots = response['DBSnapshots']
for s in snapshots:
    dest_snapshots.append(s['DBSnapshotIdentifier'])
    
print dest_snapshots

In [ ]:
for ss in source_snapshots:
    idMinusRDS = ss['id'].split(':')[1]
    if idMinusRDS not in dest_snapshots:
        print 'copy', ss

In [ ]:
targetParts = TargetDBSnapshotIdentifier=source_snapshots[0]['id'].split(':')
target = targetParts[1]
response = dest_client.copy_db_snapshot(
    SourceDBSnapshotIdentifier=source_snapshots[0]['arn'],
    TargetDBSnapshotIdentifier=target
)

print response